# Lab 3.3: Backward Propagation Implementation

## Learning Objectives
- Understand the mathematics behind backward propagation
- Implement backward propagation using vectorized operations
- Calculate gradients for weights and biases in all layers
- Verify gradient calculations using numerical methods

## Duration: 45 minutes

## Prerequisites
- Completion of Labs 3.1 and 3.2
- Understanding of chain rule in calculus
- Knowledge of forward propagation mechanics

## Setup and Environment

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_moons
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 8)

print("Environment setup complete!")
print(f"NumPy version: {np.__version__}")

## Part 1: Backward Propagation Theory (8 minutes)

### The Chain Rule in Neural Networks

Backward propagation uses the chain rule to compute gradients:

For layer l:
- **dW^[l]** = (1/m) * dZ^[l] * A^[l-1].T
- **db^[l]** = (1/m) * sum(dZ^[l], axis=1, keepdims=True)
- **dA^[l-1]** = W^[l].T * dZ^[l]

Where:
- **dZ^[l]** = dA^[l] * g'^[l](Z^[l])
- g'^[l] is the derivative of the activation function

In [ ]:
# Copy the activation functions from Lab 3.2
class ActivationFunctions:
    """Collection of activation functions and their derivatives"""
    
    @staticmethod
    def relu(z):
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        return (z > 0).astype(float)
    
    @staticmethod
    def sigmoid(z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    @staticmethod
    def sigmoid_derivative(z):
        s = ActivationFunctions.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def tanh(z):
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        return 1 - np.tanh(z)**2
    
    @staticmethod
    def linear(z):
        return z
    
    @staticmethod
    def linear_derivative(z):
        return np.ones_like(z)

print("Activation functions loaded!")

## Part 2: Cost Functions (7 minutes)

Before implementing backward propagation, we need cost functions to compute initial gradients:

In [ ]:
class CostFunctions:
    """Cost functions and their derivatives"""
    
    @staticmethod
    def binary_cross_entropy(AL, Y):
        """
        Binary cross-entropy cost function
        
        Parameters:
        AL: predictions (1, m)
        Y: true labels (1, m)
        
        Returns:
        cost: scalar cost
        """
        m = Y.shape[1]
        
        # Avoid log(0) by clipping predictions
        AL = np.clip(AL, 1e-15, 1 - 1e-15)
        
        cost = -1/m * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
        cost = np.squeeze(cost)  # Remove extra dimensions
        
        return cost
    
    @staticmethod
    def binary_cross_entropy_derivative(AL, Y):
        """
        Derivative of binary cross-entropy
        
        Returns:
        dAL: gradient with respect to final layer activations
        """
        # Avoid division by zero
        AL = np.clip(AL, 1e-15, 1 - 1e-15)
        
        dAL = -(Y / AL) + (1 - Y) / (1 - AL)
        
        return dAL
    
    @staticmethod
    def mean_squared_error(AL, Y):
        """
        Mean squared error cost function
        """
        m = Y.shape[1]
        cost = 1/(2*m) * np.sum(np.square(AL - Y))
        return cost
    
    @staticmethod
    def mean_squared_error_derivative(AL, Y):
        """
        Derivative of mean squared error
        """
        m = Y.shape[1]
        dAL = 1/m * (AL - Y)
        return dAL

# Test cost functions
# Generate test data
test_predictions = np.array([[0.8, 0.2, 0.9, 0.1]])
test_labels = np.array([[1, 0, 1, 0]])

cost = CostFunctions.binary_cross_entropy(test_predictions, test_labels)
dAL = CostFunctions.binary_cross_entropy_derivative(test_predictions, test_labels)

print("Cost Functions Test:")
print(f"Predictions: {test_predictions.flatten()}")
print(f"True labels: {test_labels.flatten()}")
print(f"Binary cross-entropy cost: {cost:.4f}")
print(f"Cost derivative: {dAL.flatten()}")
print("Cost functions implemented successfully!")

## Part 3: Backward Propagation Implementation (20 minutes)

Now let's implement the complete backward propagation algorithm:

In [ ]:
class BackwardPropagation:
    """
    Complete backward propagation implementation
    """
    
    def __init__(self):
        self.activations = ActivationFunctions()
        self.cost_functions = CostFunctions()
    
    def linear_backward(self, dZ, cache):
        """
        Linear portion of backward propagation for one layer
        
        Parameters:
        dZ: gradient of cost with respect to linear output of layer l
        cache: tuple of (A_prev, W, b) from forward propagation
        
        Returns:
        dA_prev: gradient of cost with respect to activation of layer l-1
        dW: gradient of cost with respect to weights of layer l
        db: gradient of cost with respect to bias of layer l
        """
        A_prev, W, b = cache
        m = A_prev.shape[1]
        
        # Compute gradients
        dW = 1/m * np.dot(dZ, A_prev.T)
        db = 1/m * np.sum(dZ, axis=1, keepdims=True)
        dA_prev = np.dot(W.T, dZ)
        
        return dA_prev, dW, db
    
    def linear_activation_backward(self, dA, cache, activation):
        """
        Backward propagation for one layer (linear + activation)
        
        Parameters:
        dA: gradient of cost with respect to activation of layer l
        cache: tuple of (linear_cache, activation_cache)
        activation: activation function name
        
        Returns:
        dA_prev: gradient of cost with respect to activation of layer l-1
        dW: gradient of cost with respect to weights of layer l
        db: gradient of cost with respect to bias of layer l
        """
        linear_cache, activation_cache = cache
        
        # Compute dZ based on activation function
        if activation == 'relu':
            dZ = dA * self.activations.relu_derivative(activation_cache)
        elif activation == 'sigmoid':
            dZ = dA * self.activations.sigmoid_derivative(activation_cache)
        elif activation == 'tanh':
            dZ = dA * self.activations.tanh_derivative(activation_cache)
        elif activation == 'linear':
            dZ = dA * self.activations.linear_derivative(activation_cache)
        else:
            raise ValueError(f"Unsupported activation function: {activation}")
        
        # Linear backward
        dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        
        return dA_prev, dW, db
    
    def backward_propagation(self, AL, Y, caches, activation_functions, cost_function='binary_cross_entropy'):
        """
        Complete backward propagation for the entire network
        
        Parameters:
        AL: probability vector, output of forward propagation (output_size, m)
        Y: true "label" vector (output_size, m)
        caches: list of caches from forward propagation
        activation_functions: list of activation functions for each layer
        cost_function: name of cost function to use
        
        Returns:
        gradients: dictionary with gradients for each parameter
        """
        gradients = {}
        L = len(caches)  # Number of layers
        Y = Y.reshape(AL.shape)  # Ensure Y has same shape as AL
        
        # Initialize backward propagation
        if cost_function == 'binary_cross_entropy':
            dAL = self.cost_functions.binary_cross_entropy_derivative(AL, Y)
        elif cost_function == 'mean_squared_error':
            dAL = self.cost_functions.mean_squared_error_derivative(AL, Y)
        else:
            raise ValueError(f"Unsupported cost function: {cost_function}")
        
        # Backward propagation through all layers
        dA = dAL
        
        for l in reversed(range(L)):
            cache = caches[l]
            activation = activation_functions[l]
            
            dA_prev, dW, db = self.linear_activation_backward(dA, cache, activation)
            
            # Store gradients
            gradients[f'dW{l+1}'] = dW
            gradients[f'db{l+1}'] = db
            
            # Update dA for next iteration
            dA = dA_prev
        
        return gradients
    
    def compute_cost(self, AL, Y, cost_function='binary_cross_entropy'):
        """
        Compute the cost
        """
        if cost_function == 'binary_cross_entropy':
            return self.cost_functions.binary_cross_entropy(AL, Y)
        elif cost_function == 'mean_squared_error':
            return self.cost_functions.mean_squared_error(AL, Y)
        else:
            raise ValueError(f"Unsupported cost function: {cost_function}")

print("BackwardPropagation class implemented successfully!")

## Part 4: Integration with Forward Propagation (5 minutes)

Let's combine forward and backward propagation in a complete neural network:

In [ ]:
# Import/recreate the forward propagation from Lab 3.2
class NeuralNetwork:
    """
    Complete neural network with forward and backward propagation
    """
    
    def __init__(self):
        self.activations = ActivationFunctions()
        self.backward_prop = BackwardPropagation()
    
    def initialize_parameters(self, layer_dims):
        """Initialize weights and biases"""
        parameters = {}
        
        for l in range(1, len(layer_dims)):
            # Xavier initialization
            parameters[f'W{l}'] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2.0 / layer_dims[l-1])
            parameters[f'b{l}'] = np.zeros((layer_dims[l], 1))
        
        return parameters
    
    def forward_propagation(self, X, parameters, activation_functions):
        """Forward propagation"""
        caches = []
        A = X
        L = len(parameters) // 2
        
        for l in range(1, L + 1):
            A_prev = A
            W = parameters[f'W{l}']
            b = parameters[f'b{l}']
            
            # Linear forward
            Z = np.dot(W, A_prev) + b
            linear_cache = (A_prev, W, b)
            
            # Activation forward
            activation = activation_functions[l-1]
            if activation == 'sigmoid':
                A = self.activations.sigmoid(Z)
            elif activation == 'relu':
                A = self.activations.relu(Z)
            elif activation == 'tanh':
                A = self.activations.tanh(Z)
            elif activation == 'linear':
                A = self.activations.linear(Z)
            
            cache = (linear_cache, Z)
            caches.append(cache)
        
        return A, caches
    
    def compute_gradients(self, X, Y, parameters, activation_functions, cost_function='binary_cross_entropy'):
        """
        Compute gradients using forward and backward propagation
        
        Returns:
        cost: computed cost
        gradients: gradients for all parameters
        """
        # Forward propagation
        AL, caches = self.forward_propagation(X, parameters, activation_functions)
        
        # Compute cost
        cost = self.backward_prop.compute_cost(AL, Y, cost_function)
        
        # Backward propagation
        gradients = self.backward_prop.backward_propagation(AL, Y, caches, activation_functions, cost_function)
        
        return cost, gradients, AL

print("Complete NeuralNetwork class created!")

## Part 5: Testing Backward Propagation (5 minutes)

### Test 1: Simple Binary Classification

In [ ]:
# Create a simple test case
nn = NeuralNetwork()

# Network architecture
layer_dims = [2, 3, 1]  # 2 inputs -> 3 hidden -> 1 output
activation_functions = ['relu', 'sigmoid']

# Initialize parameters
parameters = nn.initialize_parameters(layer_dims)

# Create test data
X_test = np.array([[1.5, 2.0], [0.5, -1.0], [2.5, 1.5], [-0.5, 0.8]]).T  # (2, 4)
Y_test = np.array([[1, 0, 1, 0]])  # (1, 4)

print("Test Setup:")
print(f"Network: {layer_dims}")
print(f"Activations: {activation_functions}")
print(f"Input shape: {X_test.shape}")
print(f"Labels shape: {Y_test.shape}")
print(f"Test data:\nX =\n{X_test}")
print(f"Y = {Y_test.flatten()}")

# Compute gradients
cost, gradients, predictions = nn.compute_gradients(X_test, Y_test, parameters, activation_functions)

print(f"\nResults:")
print(f"Cost: {cost:.6f}")
print(f"Predictions: {predictions.flatten()}")
print("\nGradient shapes:")
for key, grad in gradients.items():
    print(f"{key}: {grad.shape}")

print("\nGradient values:")
for key, grad in gradients.items():
    print(f"{key}:")
    print(grad)
    print()

### Test 2: Gradient Checking (Numerical Verification)

In [ ]:
def gradient_check(nn, X, Y, parameters, activation_functions, epsilon=1e-7):
    """
    Gradient checking using numerical differentiation
    
    Parameters:
    epsilon: small value for numerical differentiation
    
    Returns:
    difference: relative difference between analytical and numerical gradients
    """
    # Compute analytical gradients
    cost, gradients, _ = nn.compute_gradients(X, Y, parameters, activation_functions)
    
    # Convert gradients to vector
    grad_vector = []
    param_names = []
    
    for key in sorted(gradients.keys()):
        grad_vector.extend(gradients[key].flatten())
        param_names.extend([key] * gradients[key].size)
    
    grad_vector = np.array(grad_vector)
    
    # Compute numerical gradients
    num_grad_vector = np.zeros_like(grad_vector)
    
    # Convert parameters to vector
    param_vector = []
    for l in range(1, len(parameters)//2 + 1):
        param_vector.extend(parameters[f'W{l}'].flatten())
        param_vector.extend(parameters[f'b{l}'].flatten())
    
    param_vector = np.array(param_vector)
    
    # Check only a subset of parameters for efficiency
    check_indices = np.random.choice(len(param_vector), min(20, len(param_vector)), replace=False)
    
    for i, idx in enumerate(check_indices):
        # Reconstruct parameters with theta + epsilon
        param_plus = param_vector.copy()
        param_plus[idx] += epsilon
        params_plus = vector_to_parameters(param_plus, parameters)
        
        # Reconstruct parameters with theta - epsilon
        param_minus = param_vector.copy()
        param_minus[idx] -= epsilon
        params_minus = vector_to_parameters(param_minus, parameters)
        
        # Compute costs
        AL_plus, _ = nn.forward_propagation(X, params_plus, activation_functions)
        cost_plus = nn.backward_prop.compute_cost(AL_plus, Y)
        
        AL_minus, _ = nn.forward_propagation(X, params_minus, activation_functions)
        cost_minus = nn.backward_prop.compute_cost(AL_minus, Y)
        
        # Numerical gradient
        num_grad_vector[idx] = (cost_plus - cost_minus) / (2 * epsilon)
    
    # Check only the indices we computed
    analytical_subset = grad_vector[check_indices]
    numerical_subset = num_grad_vector[check_indices]
    
    # Compute relative difference
    numerator = np.linalg.norm(analytical_subset - numerical_subset)
    denominator = np.linalg.norm(analytical_subset) + np.linalg.norm(numerical_subset)
    difference = numerator / denominator if denominator != 0 else 0
    
    return difference, analytical_subset, numerical_subset

def vector_to_parameters(param_vector, template_params):
    """Convert parameter vector back to parameter dictionary"""
    parameters = {}
    idx = 0
    
    for l in range(1, len(template_params)//2 + 1):
        W_shape = template_params[f'W{l}'].shape
        b_shape = template_params[f'b{l}'].shape
        
        W_size = np.prod(W_shape)
        b_size = np.prod(b_shape)
        
        parameters[f'W{l}'] = param_vector[idx:idx+W_size].reshape(W_shape)
        idx += W_size
        
        parameters[f'b{l}'] = param_vector[idx:idx+b_size].reshape(b_shape)
        idx += b_size
    
    return parameters

# Perform gradient check on our test case
print("Performing Gradient Check...")
difference, analytical, numerical = gradient_check(nn, X_test, Y_test, parameters, activation_functions)

print(f"\nGradient Check Results:")
print(f"Relative difference: {difference:.2e}")
print(f"Sample analytical gradients: {analytical[:5]}")
print(f"Sample numerical gradients: {numerical[:5]}")

if difference < 1e-5:
    print("✅ Gradient check PASSED! Backpropagation is correct.")
elif difference < 1e-3:
    print("⚠️ Gradient check WARNING: Small discrepancy detected.")
else:
    print("❌ Gradient check FAILED: Large discrepancy detected.")

## Progress Tracking Checklist

Check off each item as you complete it:

- [ ] **Environment Setup**: Imported libraries and configured environment
- [ ] **Theory Understanding**: Reviewed backward propagation mathematics
- [ ] **Activation Derivatives**: Implemented activation function derivatives
- [ ] **Cost Functions**: Implemented cost functions and their derivatives
- [ ] **Linear Backward**: Implemented linear portion of backward propagation
- [ ] **Activation Backward**: Implemented activation portion of backward propagation
- [ ] **Full Backward**: Implemented complete backward propagation
- [ ] **Integration**: Combined forward and backward propagation
- [ ] **Simple Test**: Tested on simple binary classification
- [ ] **Gradient Check**: Verified gradients using numerical methods
- [ ] **Lab Completion**: Successfully completed all exercises

## Key Concepts Summary

### What You've Learned:
1. **Chain Rule Application**: How calculus enables gradient computation
2. **Gradient Flow**: How gradients propagate backward through layers
3. **Vectorized Implementation**: Efficient gradient computation for multiple examples
4. **Numerical Verification**: Using gradient checking to verify implementations
5. **Cost Function Integration**: Connecting loss to gradient computation

### Mathematical Foundation:
- **Weight Gradients**: dW^[l] = (1/m) * dZ^[l] * A^[l-1].T
- **Bias Gradients**: db^[l] = (1/m) * sum(dZ^[l])
- **Activation Gradients**: dA^[l-1] = W^[l].T * dZ^[l]
- **Chain Rule**: dZ^[l] = dA^[l] * g'^[l](Z^[l])

## Validation Steps

In [ ]:
# Validation Test 1: Gradient Shapes
def test_gradient_shapes():
    """Test if gradients have correct shapes"""
    try:
        test_nn = NeuralNetwork()
        test_dims = [3, 5, 2, 1]
        test_activations = ['relu', 'tanh', 'sigmoid']
        test_params = test_nn.initialize_parameters(test_dims)
        
        X_test = np.random.randn(3, 10)
        Y_test = np.random.randint(0, 2, (1, 10))
        
        _, gradients, _ = test_nn.compute_gradients(X_test, Y_test, test_params, test_activations)
        
        # Check gradient shapes match parameter shapes
        for l in range(1, len(test_dims)):
            W_shape = test_params[f'W{l}'].shape
            b_shape = test_params[f'b{l}'].shape
            
            assert gradients[f'dW{l}'].shape == W_shape, f"dW{l} shape mismatch"
            assert gradients[f'db{l}'].shape == b_shape, f"db{l} shape mismatch"
        
        print("✅ Gradient shapes test passed!")
        return True
    except Exception as e:
        print(f"❌ Gradient shapes test failed: {e}")
        return False

test_gradient_shapes()

In [ ]:
# Validation Test 2: Cost Decreases
def test_cost_behavior():
    """Test if cost behaves correctly"""
    try:
        test_nn = NeuralNetwork()
        
        # Perfect predictions should have low cost
        Y_perfect = np.array([[1, 0, 1, 0]])
        AL_perfect = np.array([[0.99, 0.01, 0.98, 0.02]])
        cost_perfect = test_nn.backward_prop.compute_cost(AL_perfect, Y_perfect)
        
        # Random predictions should have higher cost
        AL_random = np.array([[0.5, 0.5, 0.5, 0.5]])
        cost_random = test_nn.backward_prop.compute_cost(AL_random, Y_perfect)
        
        # Wrong predictions should have highest cost
        AL_wrong = np.array([[0.01, 0.99, 0.02, 0.98]])
        cost_wrong = test_nn.backward_prop.compute_cost(AL_wrong, Y_perfect)
        
        assert cost_perfect < cost_random < cost_wrong, "Cost ordering incorrect"
        
        print(f"Perfect predictions cost: {cost_perfect:.4f}")
        print(f"Random predictions cost: {cost_random:.4f}")
        print(f"Wrong predictions cost: {cost_wrong:.4f}")
        print("✅ Cost behavior test passed!")
        return True
    except Exception as e:
        print(f"❌ Cost behavior test failed: {e}")
        return False

test_cost_behavior()

## Troubleshooting Guide

### Common Issues and Solutions:

**Issue 1: Gradient explosion/vanishing**
- **Cause**: Poor weight initialization or activation functions
- **Solution**: Use Xavier initialization, avoid sigmoid in deep networks

**Issue 2: Gradient check fails**
- **Cause**: Implementation errors in backward propagation
- **Solution**: Check activation derivatives, matrix dimensions, chain rule application

**Issue 3: Shape mismatch errors**
- **Cause**: Incorrect matrix operations
- **Solution**: Verify cache structure, gradient shapes match parameter shapes

**Issue 4: NaN or infinite values**
- **Cause**: Numerical instability in activation/cost functions
- **Solution**: Use clipping in sigmoid/log functions, check for divide by zero

**Issue 5: Slow convergence**
- **Cause**: Poor gradient computation or learning rate
- **Solution**: Verify backward propagation implementation, tune hyperparameters

### Debugging Tips:
- Use gradient checking on small networks first
- Print intermediate gradient values
- Check gradient norms (should not be too large or small)
- Verify cost decreases over iterations

## Cleanup Instructions

1. **Save your work**: Save this notebook with your implementations
2. **Clear output**: Cell → All Output → Clear (optional, saves space)
3. **Close plots**: Close any open matplotlib windows
4. **Memory cleanup**: Variables will be cleared when kernel is restarted

In [ ]:
# Final summary and cleanup
print("🎉 Lab 3.3: Backward Propagation Implementation Completed!")
print("\n📋 What you accomplished:")
print("✅ Implemented complete backward propagation algorithm")
print("✅ Created cost functions and their derivatives")
print("✅ Integrated forward and backward propagation")
print("✅ Verified implementation with gradient checking")
print("✅ Tested on real neural network architectures")
print("\n🎯 Next: Lab 3.4 - Multi-class Classification Setup")

# Optional: Clean up memory
import gc
gc.collect()
print("\n🧹 Memory cleaned up successfully!")